<a href="https://colab.research.google.com/github/ydaigo/BERT_HANDSON/blob/master/BERT_HANDS_ON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERTのハンズオン
[transformers](https://github.com/huggingface/transformers)を動かします。

In [0]:
#準備
!pip install transformers
!pip install mecab-python3
from transformers import pipeline,AutoTokenizer,BertTokenizer,AutoModelForSequenceClassification,BertJapaneseTokenizer, BertForMaskedLM

     |████████████████████████████████| 573kB 3.1MB/s 
     |████████████████████████████████| 1.0MB 8.5MB/s 
     |████████████████████████████████| 890kB 19.8MB/s 
     |████████████████████████████████| 3.7MB 23.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=a187d3e92145ab4be94466c0240a255d554add4f088a6a4d090eec56e8269a7d
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
     |████████████████████████████████| 17.1MB 239kB/s 


## トークンナイザー
トークンナイザーは文章を単語ごとに分割し数字を割り当てる。

In [0]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
sequence = "私はエンジニアである。"
tokenized_sequence = tokenizer.tokenize(sequence)
print(tokenized_sequence)
encoded_sequence = tokenizer.encode(sequence)
print(encoded_sequence)


['私', '##は', '##エンジニア', '##で', '##ある', '。']
[2, 1325, 28450, 28326, 28455, 16071, 8, 3]


#感情分類
感情判定のモデルです。  
0:ポジティブ
1:ネガティブ

[bert-base-japanese-char-whole-word-masking](https://huggingface.co/bert-base-japanese-char-whole-word-masking)というモデルを自作データセットでファインチューニングをしました。  
[ファインチューニングしたモデル](daigo/bert-base-japanese-sentiment)

[このサイト](https://qiita.com/kenta1984/items/7f3a5d859a15b20657f3)を参考にしました。

In [0]:
model = AutoModelForSequenceClassification.from_pretrained('daigo/bert-base-japanese-sentiment') 
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
nlp = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)
print(nlp("私は幸せである。"))

[{'label': 'ネガティブ', 'score': 0.7896077}]


# バンダイナムコが公開したモデルを使ってみる。
[リポジトリ](https://github.com/BandaiNamcoResearchInc/DistilBERT-base-jp?utm_campaign=Weekly%20Kaggle%20News&utm_medium=email&utm_source=Revue%20newsletter)

In [0]:
nlp = pipeline("fill-mask",model="bandainamco-mirai/distilbert-base-japanese",tokenizer="cl-tohoku/bert-base-japanese-whole-word-masking")
nlp(f"私は北陸のシステムエンジニアだ。私は機械学習を{nlp.tokenizer.mask_token}している。")

Model name 'bandainamco-mirai/distilbert-base-japanese' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, bert-base-japanese-char-whole-word-masking, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased, bart-large, bart-large-mnli, bart-large-cnn, bart-large-xsum, openai-gpt, transfo-xl-wt103, gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2, ctrl, xlnet-base-cased, xlnet-large-cased, xlm-mlm-en-2048, xlm-mlm-ende-1024, xlm-mlm-enfr-1024, xl

[{'score': 0.14600779116153717,
  'sequence': '[CLS] 私は北陸のシステムエンジニアだ 。 私は機械学習を 勉強 している 。 [SEP]',
  'token': 8192},
 {'score': 0.04847852140665054,
  'sequence': '[CLS] 私は北陸のシステムエンジニアだ 。 私は機械学習を 経験 している 。 [SEP]',
  'token': 1903},
 {'score': 0.04197195544838905,
  'sequence': '[CLS] 私は北陸のシステムエンジニアだ 。 私は機械学習を 独学 している 。 [SEP]',
  'token': 24789},
 {'score': 0.03059275448322296,
  'sequence': '[CLS] 私は北陸のシステムエンジニアだ 。 私は機械学習を 受験 している 。 [SEP]',
  'token': 6644},
 {'score': 0.02952825464308262,
  'sequence': '[CLS] 私は北陸のシステムエンジニアだ 。 私は機械学習を 実践 している 。 [SEP]',
  'token': 7159}]

# いろいろな調整済みの学習済みモデルを使ってみよう。
[参考にしたサイト](https://huggingface.co/transformers/usage.html)

###文章要約
モデル  
bart-large-cnn


文章はアナと雪の女王の要約  
https://www.imdb.com/title/tt2294629/plotsummary

In [0]:
from transformers import pipeline
text = """When the newly crowned Queen Elsa accidentally uses her power to turn things into ice to curse her home in infinite winter,
           her sister Anna teams up with a mountain man, his playful reindeer, and a snowman to change the weather condition."""
summarizer = pipeline("summarization",model="bart-large-cnn")
summarizer(text, min_length=5, max_length=30)

Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-cnn/modelcard.json' to download model card file.
Creating an empty model card.


[{'summary_text': 'Queen Elsa accidentally uses her power to turn things into ice to curse her home in infinite winter. Her sister Anna teams up with a mountain'}]

###質問解答
モデル  
[bert-large-uncased-whole-word-masking-finetuned-squad](https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad)  
トークンナイザー  
[bert-base-uncased](https://huggingface.co/bert-base-uncased)

In [0]:
from transformers import pipeline

nlp = pipeline("question-answering",model="distilbert-base-cased-distilled-squad")

context = text
print(nlp(question="Who is Queen?", context=context))

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1935.53it/s]


{'score': 0.6697545077909517, 'start': 29, 'end': 33, 'answer': 'Elsa'}


###文章生成  
モデル
[gpt2](https://huggingface.co/gpt2#)

In [0]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")

sequence = f"I am System Engineer. But"

input = tokenizer.encode(sequence, return_tensors="pt")
generated = model.generate(input, max_length=500, do_sample=True)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


I am System Engineer. But I am also an engineer.

For people with different experiences we should have a clear definition of "skill", a phrase used to describe the skills that are necessary to succeed, such as the skill of helping others and working with one another. The term job description is used interchangeably here, but if that were the position someone would have, I'd be interested in helping others make the mistakes that are necessary. One who doesn't have a strong understanding of these skills but, to their credit, is not one of those people who should be able to be "job trained".

The term skill is typically used to describe a skill as being an aspect of one's working skill set.

My primary passion for programming and software engineering was to help people in their personal lives use their understanding of human-computer interaction to help them solve complex problems using a computer based software. In this manner I developed a strong commitment to giving the people I helped